Import

In [ ]:
# Please restart the kernel and run all cells after executing this cell.
!pip install --user "ibm-watsonx-ai==1.1.2" | tail -n 1
!pip install --user "langchain==0.2.1" | tail -n 1
!pip install --user "langchain-ibm==0.1.11" | tail -n 1
!pip install --user "langchain-community==0.2.1" | tail -n 1
#!pip install --user "chromadb==0.4.24" | tail -n 1
!pip install --user "chromadb==0.3.29" | tail -n 1
!pip install --user "pypdf==4.3.1" | tail -n 1
!pip install --user "lark==1.1.9" | tail -n 1

Helper function

In [1]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

Build LLM

In [ ]:
import 

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models.extensions.langchain import WatsonxLLM

In [30]:
import os

os.environ['WATSONX_APIKEY'] = 'replace IBM API key'
os.environ['WATSONX_PROJECTID'] = 'replace IBM project ID'
#'Replace with IBM key''

In [31]:
def llm():
    model_id = 'mistralai/mixtral-8x7b-instruct-v01'
    
    parameters = {
        GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
        GenParams.TEMPERATURE: 0.5, # this randomness or creativity of the model's responses
    }
    
    credentials = {
        'apikey': os.environ['WATSONX_APIKEY'],
        'url': "https://eu-de.ml.cloud.ibm.com",
    }
    
    
    project_id = os.environ['WATSONX_PROJECTID']

      
    model = ModelInference(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    
    mixtral_llm = WatsonxLLM(model = model)
    return mixtral_llm

In [32]:
llmMQr = llm()

Text Splitter

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def text_splitter(data, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

Embedding model

In [34]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

def watsonx_embedding():
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
    }
    
    watsonx_embedding = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr",
        url="https://eu-de.ml.cloud.ibm.com",
        project_id="7ce63394-6c4b-4e32-8909-456031209bab",
        params=embed_params,
    )
    return watsonx_embedding

Retrievers

Vector Store-Backed Retriever

In [35]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt"
from langchain_community.document_loaders import TextLoader
loader = TextLoader("companypolicies.txt")
txt_data = loader.load()
chunks_txt = text_splitter(txt_data, 200, 20)

Store in Chroma DB

In [36]:
#Store in chroma db
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(chunks_txt, watsonx_embedding())

Simple similarity search

In [37]:
query = "email policy"
retriever = vectordb.as_retriever()
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization recognizes the importance of fairness and consistency in these processes, and decisions will be made after'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy aims to maintain a safe, healthy, and productive workplace.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='will ensure its alignment with evolving legal requirements and best practices.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Review of Policy: This policy will be reviewed periodically to ensure its alignment with evolving legal requirements and best practices for maintaining a healthy and safe workplace.')]

In [38]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization recognizes the importance of fairness and consistency in these processes, and decisions will be made after')]

MMR retrieval

In [39]:
retriever = vectordb.as_retriever(search_type="mmr")
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization recognizes the importance of fairness and consistency in these processes, and decisions will be made after'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Employee Referrals: We encourage and appreciate employee referrals as they contribute to building a strong and engaged team.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='harassment. This organization considers it a collective responsibility to ensure a workplace free from discrimination and harassment, and it is essential that every individual within the organization'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.')]

Similarity score threshold retrieval

In [40]:
retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.4}
)
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization recognizes the importance of fairness and consistency in these processes, and decisions will be made after'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy aims to maintain a safe, healthy, and productive workplace.')]

Multi-Query Retriever : The MultiQueryRetriever addresses this by using an LLM to generate multiple queries from different perspectives for a given user input query. 

In [41]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf"
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("langchain-paper.pdf")
pdf_data = loader.load()


Split document and store 

In [42]:
# Split
chunks_pdf = text_splitter(pdf_data, 500, 20)

# VectorDB
ids = vectordb.get()["ids"]
vectordb.delete(ids) # We need to delete existing embeddings from previous documents and then store current document embeddings in.
vectordb = Chroma.from_documents(documents=chunks_pdf, embedding=watsonx_embedding())

Multi retriever : create simila queries

In [43]:
from langchain.retrievers.multi_query import MultiQueryRetriever

query = "What does the paper say about langchain?"

retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm()
)

In [44]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [45]:
docs = retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the content of the document regarding Langchain?', '2. Can you provide a summary of the information presented in the paper about Langchain?', '3. What are the key findings in the paper related to Langchain?']


[Document(metadata={'source': 'langchain-paper.pdf', 'page': 3}, page_content='Step 8.  User Response Delivery:  Present the model -\ngenerated response to the user, thereby delivering the \nmental health advice or information they sought .\n \nFigure 3. MindGuide Chatbot Architecture'),
 Document(metadata={'source': 'langchain-paper.pdf', 'page': 1}, page_content='• content: This represents the actual text or content of \nthe message. It can be a user query, a system \ninstruction, or any other relevant information.  \n• role: This represents the role or source of the \nmessage. It defines who is speaking or generating \nthe message. Common roles include "User", \n"Assistant", "System", or any other custom role you \ndefine . \nThe chat model interface is based around messages rather \nthan raw text. The types of messages supported in LangChain'),
 Document(metadata={'source': 'langchain-paper.pdf', 'page': 2}, page_content='simple task. What is less obvious are the data \nstructures 

From the log results, you can see that the LLM generated three additional queries from different perspectives based on the given query.

Self-Querying Retriever : A Self-Querying Retriever, as the name suggests, has the ability to query itself

In [46]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from lark import lark

In [47]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [48]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [49]:
vectordb = Chroma.from_documents(docs, watsonx_embedding())


In [50]:
document_content_description = "Brief summary of a movie."

retriever = SelfQueryRetriever.from_llm(
    llm(),
    vectordb,
    document_content_description,
    metadata_field_info,
)

In [51]:
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(metadata={'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [52]:
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them')]

In [53]:
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(metadata={'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

Parent Document Retriever

In [54]:
from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import CharacterTextSplitter
from langchain.storage import InMemoryStore

In [55]:
# Set two splitters. One is with big chunk size (parent) and one is with small chunk size (child)
parent_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=20, separator='\n')
child_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=20, separator='\n')

In [56]:
vectordb = Chroma(
    collection_name="split_parents", embedding_function=watsonx_embedding()
)

# The storage layer for the parent documents
store = InMemoryStore()

In [57]:
retriever = ParentDocumentRetriever(
    vectorstore=vectordb,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [58]:
retriever.add_documents(chunks_txt)

In [60]:
#Number for large chunks
len(list(store.yield_keys()))

122

In [ ]:
#Retrieving smaller chunk
sub_docs = vectordb.similarity_search("smoking policy")
print(sub_docs[0].page_content)

Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to


In [ ]:
retrieved_docs = retriever.invoke("smoking policy")
print(retrieved_docs[0].page_content)

Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to
